# Guided backpropagation

- Theoretical background: https://arxiv.org/abs/1412.6806
- API docs: https://captum.ai/api/guided_backprop.html

In [2]:
import torch
import scvi
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

Global seed set to 0


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.3 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


## load model

In [3]:
hlca_path = '../datasets/hlca.h5ad'
adata = sc.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 584944 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

In [4]:
model = scvi.model.SCANVI.load('../models/scanvi_model/', adata)
model

INFO     File ../models/scanvi_model/model.pt already downloaded                                                   


/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/model/base/_utils.py:134: UserWarning: var_names for adata passed in does not match var_names of adata used to train the model. For valid results, the vars need to be the same and in the same order as the adata used to train the model.
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(


ScanVI Model with the following params: 
unlabeled_category: unlabeled, n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: nb
Training status: Trained

## Applying feature permutation

### Measure against classification probabilities

In [5]:
from captum.attr import visualization as viz
from captum.attr import GuidedBackprop

In [7]:
gbp = GuidedBackprop(model.module)

In [8]:
n_cells = 10
scdl = model._make_data_loader(adata=adata,
                               indices=list(range(n_cells)),
                               batch_size=100)
batch = next(scdl.__iter__())

# It requires the actual model and not just the classifier function
attr = gbp.attribute((batch['X'], batch['batch']))

/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 1 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(


TypeError: new(): invalid data type 'str'

## Collapsing into required output structure

The output that we desire is genes x cell types. Currently we have number_batches * (28 cell types * batch size) * number_genes and this 14 times for the 14 datasets.

In [32]:
# 1. Collapse datasets attributions into a single tuple of attributions
# 2. Collapse the batches and cell types into a single vector
# 3. Reshape into genes * cell types